# Optimization Estimations

## Problem Size

The Problem Size (N) should maximize the use of available memory while allowing for system operations and ensuring divisibility by the Block Size (NB). Given a total memory $M$ and a percentage of memory to be used $U$, the formula for $N$ can be approximated as follows, considering double precision (8 bytes per number):

$$
N = \text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) - \left(\text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) \mod NB\right)
$$

Where:
- $M$ = Total memory in Gigabytes
- $U$ = .7 to .8 (70 to 80% utilization)
- $NB$ = Block Size

This formula is designed to calculate the optimal size of the problem (N) that the High-Performance Linpack (HPL) benchmark can handle on a system with a specific amount of memory, taking into consideration both the total memory available and a specified utilization percentage. Let's break down the intuition behind each part of the formula:

The initial part of the formula,

$$
\sqrt{\frac{M \times U \times 10^{9}}{8}}
$$

calculates the size of the problem that can fit into the memory specified by $M$ (total memory in GiB) and $U$ (the fraction of total memory to be used). The factor $10^{9}$ converts GiB to bytes, and the divisor $8$ accounts for each double precision number consuming 8 bytes of memory. We take the square root because N is the dimension of one side of a matrix so we want a number that when squared gives us the total.

The floor function,

$$
\text{floor}(...)
$$

is used to ensure that the resulting problem size is an integer. Since we're dealing with discrete units (bytes, elements of the matrix), we can't have a fraction of an element.

The final adjustment,

$$
\text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) \mod NB
$$

ensures that the calculated problem size is divisible by the block size (NB). We want our matrix to be chunked up into even blocks and if this number isn't divisible by block size you end up with a partial matrix that must be solved.

So, the subtraction of

$$
- \left(\text{floor}\left(\sqrt{\frac{M \times U \times 10^{9}}{8}}\right) \mod NB\right)
$$

from the floor of the square root of the memory calculation ensures that the final problem size (N) is both a realistic representation of what the system's memory can handle and is perfectly divisible by the chosen block size.

## Block Size (NB)

Block size is largely affected by computer architecture in combination with the matrix size and shape. There isn't a good way to precisely estimate this so you largely will check this value via experimentation. Good values typically range from 100-256. It is common to start with:

$$
NB = 192
$$

## Process Grid (P x Q)

The Process Grid dimensions should take into account the total number of cores and aim to distribute work evenly while considering physical architecture (like NUMA nodes). For a total of $C$ cores, the goal is to find $P$ and $Q$ such that $P \times Q = C$ and $|P - Q|$ is minimized. An initial approximation can be to set $P$ and $Q$ to the square root of $C$:

$$
P = Q = \sqrt{C}
$$

To be more exact, we have to make sure P and Q are whole numbers and generally P should be smaller than Q.

$$
P = 2 \times \lfloor \frac{\sqrt{C}}{2} \rfloor
$$

$$
Q = \left\{
    \begin{array}{ll}
        P & \text{if } P \times P \geq C \\
        P + 2 & \text{otherwise}
    \end{array}
\right.
$$

- **$ \sqrt{C} $**: The square root of the total number of cores ($C$) gives an initial estimate for the dimensions of a square process grid that would, in theory, equally distribute the workload across all cores. This is a common starting point for determining $P$ and $Q$ because it simplifies the assumption of an even distribution of tasks.
- **$2 \times \lfloor \frac{\sqrt{C}}{2} \rfloor$**: This calculation adjusts the initial estimate to ensure $P$ is an even number. We divide the initial estimate by $2$, round down to the nearest whole number (using the floor function, $\lfloor \cdot \rfloor$), and then multiply by $2$ again. This gets you the nearest whole number.
- **$P$ or $P + 2$**: The conditional logic for $Q$ addresses the total coverage of cores while also adhering to the constraint that $P \leq Q$ and both are even. The choice between $P$ and $P + 2$ ensures that:
  - $Q$ is at least as large as $P$, maintaining an even distribution of tasks.
  - If simply doubling $P$ does not cover all cores ($P \times P < C$), $Q$ is incremented by $2$ (to the next even number) to help cover the shortfall.
  - This adjustment maintains evenness and aims to utilize as many cores as possible without exceeding the total number of available cores. It also respects the preference for $P$ being smaller than $Q$ if they are not equal, which can be relevant for optimizing certain types of parallel computation that perform better with a "taller" rather than a "wider" grid.

**WARNING** This does not account for NUMA.

